In [3]:
import pandas as pd

laptops = pd.read_csv("laptops.csv", encoding="Windows-1251")

def clean_col(string):
    string = string.strip()
    string = string.replace("Operating System", "os")
    string = string.replace(" ", "_")
    string = string.replace("(","")
    string = string.replace(")","")
    string = string.lower()
    return string

#print(laptops.columns)

new_columns = []

for col1 in laptops.columns:
    new_c = clean_col(col1)
    new_columns.append(new_c)
    
laptops.columns = new_columns

print(laptops)

laptops["screen_size"] = laptops["screen_size"].str.replace('"','').astype(float)
laptops.rename({"screen_size": "screen_size_inches"}, axis=1, inplace=True)

laptops["ram"] = laptops["ram"].str.replace('GB','').astype(int)

laptops.rename({"ram":"ram_gb"}, axis=1, inplace=True)

dtypes = laptops.dtypes

     manufacturer                                 model_name  \
0           Apple                                MacBook Pro   
1           Apple                                Macbook Air   
2              HP                                     250 G6   
3           Apple                                MacBook Pro   
4           Apple                                MacBook Pro   
5            Acer                                   Aspire 3   
6           Apple                                MacBook Pro   
7           Apple                                Macbook Air   
8            Asus                            ZenBook UX430UN   
9            Acer                                    Swift 3   
10             HP                                     250 G6   
11             HP                                     250 G6   
12          Apple                                MacBook Pro   
13           Dell                              Inspiron 3567   
14          Apple                       

In [4]:
laptops["screen_size"] = laptops["screen_size"].str.replace('"','').astype(float)
laptops.rename({"screen_size": "screen_size_inches"}, axis=1, inplace=True)

laptops["ram"] = laptops["ram"].str.replace('GB','').astype(int)

laptops.rename({"ram":"ram_gb"}, axis=1, inplace=True)


dtypes = laptops.dtypes

print(dtypes)

KeyError: 'screen_size'

In [6]:
laptops["weight"] = (laptops["weight"]
                        .str.replace('kgs','')
                    )
laptops["weight"] = (laptops["weight"]
                        .str.replace('kg','')
                        .astype(float)
                    )


laptops.rename({"weight":"weight_kg"}, axis=1, inplace=True)

laptops["price_euros"] = (laptops["price_euros"]
                             .str.replace(',','.')
                             .astype(float))

weight_describe = laptops["weight_kg"].describe()

price_describe = laptops["price_euros"].describe()

In [7]:
laptops["gpu_manufacturer"] = (laptops["gpu"]
                                    .str.split(n=1,expand=True)
                                    .iloc[:,0]
                               )
laptops["cpu_manufacturer"] = (laptops["cpu"]
                                    .str.split(n=1, expand=True)
                                    .iloc[:,0])


In [8]:
screen_res = laptops["screen"].str.rsplit(n=1, expand=True) ## Expand will create series into Dataframe
screen_res.columns = ["A", "B"]
screen_res.loc[screen_res["B"].isnull(), "B"] = screen_res["A"]
laptops["screen_resolution"] = screen_res["B"]

laptops_cpu = laptops["cpu"].str.replace("GHz","")

In [9]:
print(laptops.iloc[0,0:])

manufacturer                                       Apple
model_name                                   MacBook Pro
category                                       Ultrabook
screen_size_inches                                  13.3
screen                IPS Panel Retina Display 2560x1600
cpu                                 Intel Core i5 2.3GHz
ram_gb                                                 8
storage                                        128GB SSD
gpu                         Intel Iris Plus Graphics 640
os                                                 macOS
os_version                                           NaN
weight_kg                                           1.37
price_euros                                      1339.69
gpu_manufacturer                                   Intel
cpu_manufacturer                                   Intel
screen_resolution                              2560x1600
Name: 0, dtype: object


In [10]:
list(laptops)

['manufacturer',
 'model_name',
 'category',
 'screen_size_inches',
 'screen',
 'cpu',
 'ram_gb',
 'storage',
 'gpu',
 'os',
 'os_version',
 'weight_kg',
 'price_euros',
 'gpu_manufacturer',
 'cpu_manufacturer',
 'screen_resolution']

In [11]:
screen_res = laptops["screen"].str.rsplit(n=1, expand=True)
screen_res.columns = ["A", "B"]
screen_res.loc[screen_res["B"].isnull(), "B"] = screen_res["A"]
laptops["screen_resolution"] = screen_res["B"]

cpu_speed_ghz = laptops["cpu"].str.replace("GHz","")

cpu_speed_ghz = cpu_speed_ghz.str.rsplit(n=1, expand=True)

cpu_speed_ghz = cpu_speed_ghz.iloc[:,1]

laptops["cpu_speed_ghz"] = cpu_speed_ghz.astype(float)

print(laptops["cpu_speed_ghz"].unique())

[2.3  1.8  2.5  2.7  3.1  3.   2.2  1.6  2.   2.8  1.2  2.9  2.4  1.44
 1.5  1.9  1.1  1.3  2.6  3.6  3.2  1.   2.1  0.9  1.92]


In [12]:
mapping_dict = {
    'Android': 'Android',
    'Chrome OS': 'Chrome OS',
    'Linux': 'Linux',
    'Mac OS': 'macOS',
    'No OS': 'No OS',
    'Windows': 'Windows',
    'macOS': 'macOS'
}

laptops["os"] = laptops["os"].map(mapping_dict)
 

In [13]:
#drop null values from dataframe - rows(axis=0) or col(axis=1)

laptops_no_null_rows = laptops.dropna()
laptops_no_null_cols = laptops.dropna(axis=1)

## value_counts_before = laptops.loc[laptops["os_version"].isnull(), "os"].value_counts()
laptops.loc[laptops["os"] == "macOS", "os_version"] = "X"

laptops.loc[laptops["os"] == "No OS", "os_version"] = "Version Unknown" #Assign version unknown to os_version where OS is "NO OS"

value_counts_after = laptops.loc[laptops["os_version"].isnull(), "os"].value_counts()

print(laptops["os_version"].value_counts(dropna=False))

In [15]:
laptops["storage"] = laptops["storage"].str.replace('GB','').str.replace('TB','000')

laptops[["storage_1","storage_2"]] = laptops["storage"].str.split("+", expand=True)

for s in ["storage_1","storage_2"]:
    s_capacity = s + "_capacity_gb"
    s_type = s + "_type"
    laptops[[s_capacity,s_type]] = laptops[s].str.split(n=1, expand=True)
    laptops[s_capacity] = laptops[s_capacity].astype(float)
    laptops[s_type] = laptops[s_type].str.strip()
    
laptops.drop(["storage","storage_1","storage_2"], axis=1, inplace=True)

In [2]:
laptops_dtypes = laptops.dtypes
cols = ['manufacturer', 'model_name', 'category', 'screen_size_inches',
        'screen', 'cpu', 'cpu_manufacturer', 'screen_resolution', 'cpu_speed_ghz', 'ram_gb',
        'storage_1_type', 'storage_1_capacity_gb', 'storage_2_type',
        'storage_2_capacity_gb', 'gpu', 'gpu_manufacturer', 'os',
        'os_version', 'weight_kg', 'price_euros']

laptops = laptops[cols]

laptops.to_csv('laptops_cleaned.csv', index=False)

laptops_cleaned = pd.read_csv('laptops_cleaned.csv')

laptops_cleaned_dtypes = laptops_cleaned.dtypes


NameError: name 'laptops' is not defined

In [21]:
laptops["price_euros"].between(898,1340).index

RangeIndex(start=0, stop=1303, step=1)